In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Code_files

Mounted at /content/drive
/content/drive/MyDrive/Code_files


In [2]:
import numpy as np
import pandas as pd

Predictions for the 5 weekends between June 25th and July 31st, 2025.

In [5]:
whole_weekend = pd.read_csv('/content/drive/MyDrive/Code_files/2x2.csv', header=0, index_col=0)
display(whole_weekend)

,No rain,Rain
No rain,0.428571,0.571429
Rain,0.400000,0.600000


In [14]:
def five_week_predict(initial_state: np.array, transition_matrix: pd.DataFrame):
  predictions = [initial_state]

  for i in range(5):
    predictions.append(np.dot(predictions[-1], transition_matrix))

  return predictions


In [96]:
# Initial state: It rained during the weekend of June 20th
initial_state = np.array([0, 1])

# predictions
predictions = [initial_state]

ww_predictions = five_week_predict(initial_state, whole_weekend)

ww_display = pd.DataFrame(ww_predictions, columns=['No rain', 'Rain'])
display(ww_display)

ww_display.to_csv('/content/drive/MyDrive/Code_files/ww_display.csv')

,No rain,Rain
0,0.000000,1.000000
1,0.400000,0.600000
2,0.411429,0.588571
3,0.411755,0.588245
4,0.411764,0.588236
5,0.411765,0.588235


In [95]:
# What if it did not rain the first weekend?
initial_state = np.array([1, 0])

# predictions
predictions = [initial_state]

ww_predictions = five_week_predict(initial_state, whole_weekend)

ww_display = pd.DataFrame(ww_predictions, columns=['No rain', 'Rain'])
display(ww_display)

ww_display.to_csv('/content/drive/MyDrive/Code_files/ww_display.csv')

,No rain,Rain
0,1.000000,0.000000
1,0.428571,0.571429
2,0.412245,0.587755
3,0.411778,0.588222
4,0.411765,0.588235
5,0.411765,0.588235


Both initial states quickly stabilize to around a 58% chance of rain.

In [97]:
cal_year = pd.read_csv('/content/drive/MyDrive/Code_files/helen_matrix', header=0, index_col=0)
display(cal_year)

,No rain,Only Friday,Only Saturday,Only Sunday,Friday and Saturday,Friday and Sunday,Saturday and Sunday,Rain all weekend
No rain,0.428571,0.047619,0.047619,0.142857,0.142857,0.095238,0.000000,0.095238
Only Friday,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.666667,0.000000
Only Saturday,0.000000,0.000000,0.500000,0.250000,0.000000,0.000000,0.250000,0.000000
Only Sunday,0.600000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Friday and Saturday,0.500000,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.166667
Friday and Sunday,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
Saturday and Sunday,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000
Rain all weekend,0.400000,0.000000,0.000000,0.000000,0.200000,0.000000,0.200000,0.200000


In [98]:
# Initial state: on the weekend of June 20th, it rained on Sunday

initial_state = [0, 0, 0, 1, 0, 0, 0, 0]

cal_year_predictions = five_week_predict(initial_state, cal_year)

year_display = pd.DataFrame(cal_year_predictions, columns=['No rain', 'Only Friday', 'Only Saturday',
                                                           'Only Sunday', 'Friday and Saturday',
                                                           'Friday and Sunday', 'Saturday and Sunday',
                                                           'Rain all weekend'])
display(year_display)

year_display.to_csv('/content/drive/MyDrive/Code_files/year_display.csv')

,No rain,Only Friday,Only Saturday,Only Sunday,Friday and Saturday,Friday and Sunday,Saturday and Sunday,Rain all weekend
0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
1,0.600000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.257143,0.028571,0.028571,0.085714,0.219048,0.057143,0.266667,0.057143
3,0.482585,0.046531,0.063039,0.080385,0.086259,0.024490,0.090952,0.125760
4,0.415303,0.055134,0.068876,0.099077,0.121848,0.045960,0.090122,0.103679
5,0.416882,0.059407,0.074522,0.096856,0.121423,0.039553,0.092736,0.098621


In [70]:
one_year_actual = pd.DataFrame({'Dates': ["6/20/25 - 6/22/25", "6/27/25 - 6/29/25",
                                              "7/4/25 - 7/6/25", '7/11/25 - 7/13/25',
                                              "7/18/25 - 7/20/25", '7/25/25 - 7/27/25'],
                                   'Actual Weather': ['Only Sunday', 'Only Saturday', 'No rain',
                                                      'No rain', 'Only Sunday', 'Friday and Sunday']})

In [71]:
# find probabilities of the actual displayed weather
vals = []
for i in range(len(year_display)):
  vals.append(year_display.loc[i, one_year_actual.loc[i, 'Actual Weather']])
one_year_actual['Probability'] = vals

# find cumulative probabilities
cum_vals = [1]
for i in range(1, len(one_year_actual)):
  cum_vals.append(cum_vals[-1] * one_year_actual.loc[i, 'Probability'])
one_year_actual['Cumulative Probability'] = cum_vals


In [72]:
display(one_year_actual)

,Dates,Actual Weather,Probability,Cumulative Probability
0,6/20/25 - 6/22/25,Only Sunday,1.000000,1.0
1,6/27/25 - 6/29/25,Only Saturday,0.000000,0.0
2,7/4/25 - 7/6/25,No rain,0.257143,0.0
3,7/11/25 - 7/13/25,No rain,0.482585,0.0
4,7/18/25 - 7/20/25,Only Sunday,0.099077,0.0
5,7/25/25 - 7/27/25,Friday and Sunday,0.039553,0.0


In [81]:
# compare to predictions
one_year_predicted = one_year_actual['Dates'].to_frame()
one_year_predicted['Prediction'] = year_display.idxmax(axis=1)

In [100]:
# find probabilities of the predictions
vals = []
for i in range(len(year_display)):
  vals.append(year_display.loc[i, one_year_predicted.loc[i, 'Prediction']])
one_year_predicted['Probability'] = vals

# find cumulative probabilities
cum_vals = [1]
for i in range(1, len(one_year_actual)):
  cum_vals.append(cum_vals[-1] * one_year_predicted.loc[i, 'Probability'])
one_year_predicted['Cumulative Probability'] = cum_vals

one_year_predicted.to_csv('/content/drive/MyDrive/Code_files/one_year_predicted.csv')

display(one_year_predicted)

,Dates,Prediction,Probability,Cumulative Probability
0,6/20/25 - 6/22/25,Only Sunday,1.000000,1.000000
1,6/27/25 - 6/29/25,No rain,0.600000,0.600000
2,7/4/25 - 7/6/25,Saturday and Sunday,0.266667,0.160000
3,7/11/25 - 7/13/25,No rain,0.482585,0.077214
4,7/18/25 - 7/20/25,No rain,0.415303,0.032067
5,7/25/25 - 7/27/25,No rain,0.416882,0.013368


In [89]:
five_year = pd.read_csv('/content/drive/MyDrive/Code_files/fiveyear.csv', header=0, index_col=0)
display(five_year)

,No rain,Only Friday,Only Saturday,Only Sunday,Friday and Saturday,Friday and Sunday,Saturday and Sunday,Rain all weekend
No rain,0.341176,0.082353,0.070588,0.129412,0.094118,0.141176,0.082353,0.058824
Only Friday,0.206897,0.103448,0.137931,0.103448,0.137931,0.000000,0.206897,0.103448
Only Saturday,0.357143,0.142857,0.178571,0.142857,0.035714,0.071429,0.071429,0.000000
Only Sunday,0.406250,0.187500,0.093750,0.093750,0.093750,0.031250,0.062500,0.031250
Friday and Saturday,0.272727,0.136364,0.181818,0.181818,0.045455,0.000000,0.090909,0.090909
Friday and Sunday,0.473684,0.157895,0.157895,0.052632,0.105263,0.000000,0.052632,0.000000
Saturday and Sunday,0.233333,0.066667,0.100000,0.166667,0.033333,0.100000,0.266667,0.033333
Rain all weekend,0.333333,0.066667,0.000000,0.133333,0.133333,0.066667,0.066667,0.200000


In [91]:
# Initial state: on the weekend of June 20th, it rained on Sunday

initial_state = [0, 0, 0, 1, 0, 0, 0, 0]

five_year_predictions = five_week_predict(initial_state, five_year)

fivey_display = pd.DataFrame(five_year_predictions, columns=['No rain', 'Only Friday', 'Only Saturday',
                                                           'Only Sunday', 'Friday and Saturday',
                                                           'Friday and Sunday', 'Saturday and Sunday',
                                                           'Rain all weekend'])
display(fivey_display)

,No rain,Only Friday,Only Saturday,Only Sunday,Friday and Saturday,Friday and Sunday,Saturday and Sunday,Rain all weekend
0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
1,0.406250,0.187500,0.093750,0.093750,0.093750,0.031250,0.062500,0.031250
2,0.314335,0.107792,0.108298,0.127426,0.090035,0.075312,0.113722,0.063079
3,0.327781,0.112357,0.107975,0.126945,0.084488,0.071672,0.110568,0.058215
4,0.327407,0.111934,0.107552,0.126691,0.084937,0.072892,0.110705,0.057881
5,0.327560,0.111992,0.107656,0.126639,0.084914,0.072793,0.110660,0.057786


In [92]:
fivey_actual = pd.DataFrame({'Dates': ["6/20/25 - 6/22/25", "6/27/25 - 6/29/25",
                                              "7/4/25 - 7/6/25", '7/11/25 - 7/13/25',
                                              "7/18/25 - 7/20/25", '7/25/25 - 7/27/25'],
                                   'Actual Weather': ['Only Sunday', 'Only Saturday', 'No rain',
                                                      'No rain', 'Only Sunday', 'Friday and Sunday']})
# find probabilities of the actual displayed weather
vals = []
for i in range(len(fivey_display)):
  vals.append(fivey_display.loc[i, fivey_actual.loc[i, 'Actual Weather']])
fivey_actual['Probability'] = vals

# find cumulative probabilities
cum_vals = [1]
for i in range(1, len(fivey_actual)):
  cum_vals.append(cum_vals[-1] * fivey_actual.loc[i, 'Probability'])
fivey_actual['Full Sequence Probability'] = cum_vals

display(fivey_actual)

,Dates,Actual Weather,Probability,Cumulative Probability
0,6/20/25 - 6/22/25,Only Sunday,1.000000,1.000000
1,6/27/25 - 6/29/25,Only Saturday,0.093750,0.093750
2,7/4/25 - 7/6/25,No rain,0.314335,0.029469
3,7/11/25 - 7/13/25,No rain,0.327781,0.009659
4,7/18/25 - 7/20/25,Only Sunday,0.126691,0.001224
5,7/25/25 - 7/27/25,Friday and Sunday,0.072793,0.000089


In [93]:
# compare to predictions
fivey_predicted = fivey_actual['Dates'].to_frame()
fivey_predicted['Prediction'] = fivey_display.idxmax(axis=1)

In [94]:
# find probabilities of the predictions
vals = []
for i in range(len(fivey_display)):
  vals.append(fivey_display.loc[i, fivey_predicted.loc[i, 'Prediction']])
fivey_predicted['Probability'] = vals

# find cumulative probabilities
cum_vals = [1]
for i in range(1, len(fivey_predicted)):
  cum_vals.append(cum_vals[-1] * fivey_predicted.loc[i, 'Probability'])
fivey_predicted['Full Sequence Probability'] = cum_vals

display(fivey_predicted)

,Dates,Prediction,Probability,Cumulative Probability
0,6/20/25 - 6/22/25,Only Sunday,1.000000,1.000000
1,6/27/25 - 6/29/25,No rain,0.406250,0.406250
2,7/4/25 - 7/6/25,No rain,0.314335,0.127699
3,7/11/25 - 7/13/25,No rain,0.327781,0.041857
4,7/18/25 - 7/20/25,No rain,0.327407,0.013704
5,7/25/25 - 7/27/25,No rain,0.327560,0.004489
